<a href="https://colab.research.google.com/github/aachen6/deepTC/blob/master/deepTC_images_tracks_sync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepTC - Extraction of the Dataset

The objective of deepTC can be found on [deepTC github page](https://github.com/aachen6/deepTC/). This is the first Colab notebook for deepTC, where the best track and the satellite images of historical TCs will be extracted from website of the National Ocean and Atmosphere Agency (NOAA) and the US Naval Research Laboratory (NRL), respectively. The focus will be on the hurricanes from the Altantic Ocean and the infrared (IR) satellite images will be used.

1. **[Satellite images and tracks of TCs](https://)**

2. [Statistics of satellite images and tracks](https://)

3. [Post-binding architecture for deepTC](https://)

4. [CNN/Resnet model for deepTC](https://)

5. [GAN model for deepTC](https://)

6. [LSTM model for deepTC](https://)
 
7. [LSTM-CNN model for deepTC](https://)


Let's get started by loading some python modules, and for simplicity and efficiency due to the use of Google Colab, everthying will be extracted to and hosted on my Google Drive. A version of copy can be found on the [deepTC githubt page](https://github.com/aachen6/deepTC/). 

In [0]:
# basic module
import os
import yaml
import numpy as np
import pandas as pd
from datetime import datetime 

# web scraper
import re 
import urllib
import requests
from bs4 import BeautifulSoup as bs

# file and image management
from io import BytesIO
from zipfile import ZipFile
from PIL import Image
import matplotlib.pyplot as plt

# mount google drive
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/Colab Notebooks/deepTC'
p_data = path + os.sep + 'data/AL'
p_image = path + os.sep + 'image/AL'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Best Tracks Extraction
The best track information [HURDAT2]((https://www.nhc.noaa.gov/data/#hurdat) is available from NOAA National Hurricane Center (NHC). It's a comma-delimited, text format with six-hourly information on the location, maximum winds, central pressure, and (beginning in 2004) size of all known tropical cyclones and subtropical cyclone. The code below extracts the information, which is saved as a binary json file.

In [0]:
tracks = []
url_hurdat2 = r"https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2017-050118.txt"
response = urllib.request.urlopen(url_hurdat2)

for line in response: # loop through hurdat2 file 
  line = line.decode('utf-8').rstrip()
  temp = line.replace(" ", "").split(",")
  if len(temp)==4: # storm header - new storm evetn
    sid = temp[0]  # reorder storm keys as region, year, number, name
    storm_id = sid[0:2] + sid[4:8] + sid[2:4] + sid[8:] 
    storm_key = storm_id + temp[1]
  else: # time step of the storms
    date_str = temp[0] + temp[1]
    date = datetime.strptime(date_str, "%Y%m%d%H%M")
    [cat, lat, lon, win, pre] = temp[3:8]
    sgn = 1 if lat[-1]=="N" else -1 
    lat = sgn*float(lat[0:-1])
    sgn = 1 if lon[-1]=="E" else -1
    lon = sgn*float(lon[0:-1])
    win = float(win)
    pre = float(pre)         
    tracks.append([storm_key, date, cat, lat, lon, win, pre])
    break

f_out = p_data + os.sep + "al_hurdat2.msg"
pd_track = pd.DataFrame(tracks, columns=["storm","date","cat","lat","lon","win","pre"])
pd_track.to_msgpack(f_out)

##Satellite Image URLs Extraction
NRL hosts the historical satellite images [tcdat](https://www.nrlmry.navy.mil/tcdat/) of TCs since 1997 at a resolution of 15 minutes. The data is organized by years, region,  storm name, satellite image type (visible, infrared etc.). A function is firstly defined to extract the satellite images by year, region, and image type. BeautifulSoup is used to parse the web content and identify the image URLs. At this moment, only the URLs will be archived and images will be download later to avoid unnecessary information. 

In [0]:
# constant variable
BASE_URL = r"https://www.nrlmry.navy.mil/tcdat/"
REGIONS  = ["ATL", "CPAC", "EPAC", "WPAC"]

def bs_storm_img_by_year(year, region, img_type):
  
  o_storm_imgs = []

  # storm key
  key_region = region[0:3:2] if region=="ATL" else region[0:2]
  
  # parse storm page in specified years
  url_year   = BASE_URL + "tc{0}/".format(year)
  url_region = url_year + region + '/'
  htm_region = requests.get(url_region)
  bs_region  = bs(htm_region.content, 'html.parser')

  # loop through storms
  for a_storm in bs_region.find_all('a', href=True):
    str_storm = a_storm['href'].split("/")[0]
    if not re.match('^[0-9]+', str_storm): continue # not a storm folder
    key_base = key_region + str_storm.split(".")[0][0:2] + str_storm.split(".")[1]
    
    # check bw avail.
    url_img = url_region + str_storm + '/{0}/geo/1km_bw/'.format(img_type)  
    htm_img = requests.get(url_img)
    if htm_img.status_code!=200: # file contains no bw
      url_img = url_region + str_storm + '/{0}/geo/1km/'.format(img_type)  
      htm_img = requests.get(url_img) 
    else:
      bs_bad = bs(htm_img.content, 'html.parser')
      if bs_bad.title.string=="oops!":
        url_img = url_region + str_storm + '/{0}/geo/1km/'.format(img_type)  
        htm_img = requests.get(url_img) 

    # parse page with images 
    bs_img = bs(htm_img.content, 'html.parser')
  
    # loop through image sets 
    for a_img in bs_img.find_all('a', href=True):
      str_img = a_img['href'].split("/")[0]
      if "jpg" not in str_img: continue # not a satellite image  
      if not re.match('^[0-9]{4}', str_img): continue # not a storm 
      url_jpg   = url_img + str_img # url to the satllite image
      str_date  = str_img.split(".")[0] + str_img.split(".")[1]
      storm_key = key_base[0:2] + str_date[0:4] + key_base[2:]
      try: date = datetime.strptime(str_date, "%Y%m%d%H%M")
      except: continue 
      o_storm_imgs.append([storm_key, date, url_jpg])
      
  return o_storm_imgs


Let's now scrape the NRL satellite image archives and get all Altantic Ocean infrared satellite image URLs. The data is also saved in a binary json file.

In [0]:
region   = "ATL"   # Atlantic only 
img_type = "ir"  # use infra-red 
years    = [str(x%100).rjust(2,"0") for x in range(1997, 2018)]

storm_imgs = []
for y in years:
  print ("processing year {0}".format(y))
  _imgs = bs_storm_img_by_year(y, region, img_type)
  storm_imgs = storm_imgs + _imgs
    
f_img = p_data + os.sep + "al_ir_urls.msg"
pd_img = pd.DataFrame(storm_imgs, columns=["storm", "date", "url"])
pd_img.to_msgpack(f_img)


##Satellite Image and Best Track Sychronization

The Hurdat2 contains following classifications of the storm status:

* TD – Tropical cyclone of tropical depression intensity (< 34 knots)
* TS – Tropical cyclone of tropical storm intensity (34-63 knots)
* HU – Tropical cyclone of hurricane intensity (> 64 knots)
* EX – Extratropical cyclone (of any intensity)
* SD – Subtropical cyclone of subtropical depression intensity (< 34 knots)
* SS – Subtropical cyclone of subtropical storm intensity (> 34 knots)
* LO – A low that is neither a tropical cyclone, a subtropical cyclone, nor an extratropical cyclone (of any intensity)
* DB – Disturbance (of any intensity) 

The hurricane HU can be further classfied into five categories based on the Saffir–Simpson hurricane wind scale (SSHWS). The function below converts HU into five hurricane categories based on the maximum sustained wind speed. If the track information at next time step is passed as an argument, it also performs a check if the status of storm classification remains the same. The purpose is to expand the satellite image dataset by utilization images that occurred between 6-hour interval best track timeframe, since it has a resolution of 15 minutes.  
 

In [0]:
def saffir_simpson(info_l, info_r=None):

  wins  = np.array([64, 83, 96, 113, 137, 9999])
  
  def cat(_info):
    if _info[0]=="HU":
      diff  = wins - _info[1]
      n_cat = np.argwhere(diff>0)[0][0]
      _info[0] = "HU" + str(n_cat) 
    
    return _info

  t_info = cat(info_l)

  # if pass the track information at next time step, 
  # will check if status remains the same  
  if info_r is not None and cat(info_r)[0]!=t_info[0]:
    t_info = None  # catergories changed, return None

  return t_info


The best track information need be mapped onto the satellite images based on its occurence time. However, these two datasets are not synchronized and have different temporal resolutions with the track information at 6-hour interval while the satellite images at 15-mininute interval. To expand the datasets, satellite images fall within two consective track times with the same classfication are assigned with the same track information/classfication. This assumption in general holds since the storm system is persistent within that timeframe. For each satellite image in each storm, two consective times from the track information that bounds the image time are identifed and the storm characteristics are assigned to the image accordingly.


In [0]:
def sync_track_to_image(pd_track, pd_image):

  track_storm_keys = set(pd_track["storm"].tolist()) # storms from track
  image_storm_keys = set(pd_image["storm"].tolist()) # storms from image
  storm_keys = track_storm_keys & image_storm_keys   # common storms

  sync = [] 
  for key in storm_keys:
    print ("processing storm " + key)
    ts_in_track = pd_track[pd_track["storm"]==key]
    ts_in_image = pd_image[pd_image["storm"]==key]
    
    track_dates = ts_in_track["date"].tolist()
    track_infos = ts_in_track[["cat","win"]].values.tolist()

    for index, row in ts_in_image.iterrows():
      # check nearest track information before and after the image date 
      date = row["date"] # image date and time
      diff = np.array([(t-date).total_seconds()/3600. for t in track_dates])
      idx_0 = np.argwhere(diff==0)
      is_direct = False
      if len(idx_0)!=0: # direct match time stamp
        is_direct = True
        _info = track_infos[idx_0[0][0]]
        t_info = saffir_simpson(_info)
        _url = row["url"] 
      else: # indirect match
        try: # retain only if nearest track information has the same category 
          idx_l = np.argwhere(diff<0)[-1][0]
          idx_r = np.argwhere(diff>0)[0][0]
          t_info = saffir_simpson(track_infos[idx_l], track_infos[idx_r]) 
          if t_info is None: continue 
        except:
          continue 
          
      try: # unique image file name key_datetime.jpg
        _url = row["url"]
        _image = "{0}_{1}.jpg".format(key, row["date"].strftime("%m%d%H%M"))
      except:
        continue
 
      sync.append([key, date] + t_info + [is_direct, _url, _image])
            
  return sync  

Now, we can sychronize the dataset and archive into a binary json file.

In [0]:
# load storm tracks
f_track = r"./data/al_hurdat2.msg"
pd_track = pd.read_msgpack(f_track)
pd_track["date"] = pd.to_datetime(pd_track["date"],format="%Y-%m-%d %H%M%S")

# load storm satellite images
f_image = "./data/al_ir_urls.msg"
pd_image = pd.read_msgpack(f_image)

# sync track and image for classification labels
f_sync = "./data/al_ir_track.msg"
sync = sync_track_to_image(pd_track, pd_image)
sync.sort(key = lambda x: x[0])
pd_sync = pd.DataFrame(sync, columns=["storm","date","cat","win","direct","url","image"])
pd_sync.to_msgpack(f_sync)

##Download and Archive the Images

The function below downloads a single image with an extension to interpolate image at particular datetime based on two consective images with bounding datatime. We will skip this part since it takes a long time to download the entire image archive. The data has been downloaded locally and uploaded into the google drive.

In [0]:
import os
print (os.getcwd()) 
print ("../")

def download_blend_image(f_image, url_l, url_r=None, weight=None):

  req = requests.get(url_l, stream=True)
  try:
    img = Image.open(BytesIO(req.content))
    if url_r is not None:
      req = requests.get(url_r, stream=True)
      img_r = Image.open(BytesIO(req.content))
      img = Image.blend(img_r, img, weight)   
    img.save(f_image)
  except:
    raise ValueError("Fail to download")

/content
../
